In [2]:
import os
import json
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

def load_and_process_data(directory):
    data = []
    labels_type = []
    
    for file in os.listdir(directory):
        if file.endswith('.json'):
            with open(os.path.join(directory, file), 'r', encoding='utf-8') as f:
                json_data = json.load(f)
                
                gender = json_data['metadata.model']['metadata.model.gender']
                age = json_data['metadata.model']['metadata.model.age']
                clothes_type = json_data['metadata.clothes']['metadata.clothes.type']
                clothes_season = json_data['metadata.clothes']['metadata.clothes.season']
                
                # 나이를 숫자로 변환
                if '10대' in age:
                    age = 15
                elif '20대' in age:
                    age = 25
                elif '30대' in age:
                    age = 35
                elif '40대' in age:
                    age = 45
                elif '50대' in age:
                    age = 55
                elif '60대' in age:
                    age = 65
                else:
                    age = 0  # 나이를 알 수 없거나 범위를 벗어난 경우
                
                # 성별을 숫자로 변환
                if gender == 'FEMALE':
                    gender = 0
                elif gender == 'MALE':
                    gender = 1
                else:
                    gender = -1  # 알 수 없는 성별
                
                # 온도 데이터를 season 키값을 참조하여 설정
                if clothes_season == 'summer':
                    temperature = 25 + np.random.rand() * 10  # 25도 이상
                elif clothes_season == 'winter':
                    temperature = 5 * np.random.rand()  # 5도 이하
                else:
                    temperature = 10 + np.random.rand() * 15  # spring 또는 fall
                
                data.append([gender, age, temperature])
                labels_type.append(clothes_type)
    
    # 데이터프레임으로 변환
    df = pd.DataFrame(data, columns=['gender', 'age', 'temperature'])
    
    # 라벨 인코딩
    le_type = LabelEncoder()
    df['clothes_type'] = le_type.fit_transform(labels_type)
    
    # One-hot 인코딩
    y_type = to_categorical(df['clothes_type'])
    
    return df[['gender', 'age', 'temperature']].values, y_type, le_type

directory_path = r"Dataset\outer"
X, y_type, le_type = load_and_process_data(directory_path)


In [3]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout

# 입력 데이터의 형태
input_shape = (3,)  # 성별, 나이, 온도

# 입력층
inputs = Input(shape=input_shape)

# 은닉층
x = Dense(128, activation='relu')(inputs)
x = Dropout(0.5)(x)  # 드롭아웃 층 추가
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)  # 드롭아웃 층 추가
x = Dense(64, activation='relu')(x)
x = Dropout(0.5)(x)  # 드롭아웃 층 추가
x = Dense(64, activation='relu')(x)
x = Dropout(0.5)(x)  # 드롭아웃 층 추가

# 출력층
output_type = Dense(y_type.shape[1], activation='softmax', name='type_output')(x)

# 모델 구성
model = Model(inputs=inputs, outputs=output_type)

# 모델 컴파일
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 모델 학습
model.fit(X, y_type, epochs=20, batch_size=32)

# 모델 저장
model.save('clothing_model.h5')

# 모델 평가
results = model.evaluate(X, y_type)

# 평가 결과를 출력
print(results)


Epoch 1/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 924us/step - accuracy: 0.2542 - loss: 4.2438
Epoch 2/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 844us/step - accuracy: 0.2582 - loss: 1.7236
Epoch 3/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 818us/step - accuracy: 0.2513 - loss: 1.5071
Epoch 4/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 831us/step - accuracy: 0.2650 - loss: 1.4335
Epoch 5/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 805us/step - accuracy: 0.2882 - loss: 1.4217
Epoch 6/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 844us/step - accuracy: 0.2945 - loss: 1.4101
Epoch 7/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 814us/step - accuracy: 0.2523 - loss: 1.4141
Epoch 8/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 844us/step - accuracy: 0.2738 - loss: 1.3998
Epoch 9/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 831us/step - accuracy: 0.2766 - loss: 1.3836
Epoch 10/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 897us/step - accuracy: 0.2966 - loss: 1.3828
Epoch 11/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 858us/step - accuracy: 0.2930 - loss: 1.3913
Epoch 12/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 831us/step

77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 521us/step - accuracy: 0.2958 - loss: 1.3533
[1.3596528768539429, 0.29418936371803284]


In [11]:
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder

# 라벨 인코더를 로드하기 위해 학습 데이터의 클래스들로 인코더 재생성
def create_label_encoders(type_classes):
    le_type = LabelEncoder()
    le_type.fit(type_classes)
    return le_type

# 입력 데이터를 받아서 예측하는 함수
def predict_clothes_type(model, le_type, gender, age, temperature):
    # 입력 데이터를 넘파이 배열로 변환
    input_data = np.array([[gender, age, temperature]])
    
    # 예측 수행
    predictions = model.predict(input_data)
    
    # 예측 결과를 라벨로 변환
    predicted_type = np.argmax(predictions, axis=1)
    
    predicted_type_label = le_type.inverse_transform(predicted_type)[0]
    
    return predicted_type_label

# 모델 로드
model_path = 'clothing_model.h5'
model = load_model(model_path)

# 학습 데이터로부터 라벨 인코더 생성 (여기서는 클래스 이름들을 수동으로 입력해야 합니다)
type_classes = ['01outer_01coat', '01outer_02jacket', '01outer_04cardigan','01outer_03jumper ']  # 실제 데이터 클래스 이름들을 여기에 나열

le_type = create_label_encoders(type_classes)

# 새로운 입력 데이터를 받아 예측
gender = 1  # FEMALE
age = 20  # 예시 나이
temperature = 11  # 예시 온도

predicted_type = predict_clothes_type(model, le_type, gender, age, temperature)

print(f"Predicted Type: {predicted_type}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Predicted Type: 01outer_03jumper 
